In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.datasets import imdb

Train a LSTM on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage 
compared to simpler, much faster methods such as TF-IDF+LogReg.
Notes: 
- RNNs are tricky. Choice of batch size is important, 
choice of loss and optimizer is critical, etc. 
Most configurations won't converge.
- LSTM loss decrease during training can be quite different 
from what you see with CNNs/MLPs/etc. It's more or less a sigmoid
instead of an inverse exponential.
GPU command:
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python imdb_lstm.py
250s/epoch on GPU (GT 650M), vs. 400s/epoch on CPU (2.4Ghz Core i7).

In [3]:
max_features=20000
maxlen = 100 # cut texts after this number of words (among top max_features most common words)
batch_size = 16

In [18]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features, test_split=0.2)
print len(X_train), 'train sequences'
print len(X_test), 'test sequences' 

20000 train sequences
5000 test sequences


In [19]:
print X_train[0]
print len(X_train[0])
print y_train[0]

[17, 10, 2, 257, 7, 25, 18, 69, 4195, 1513, 16, 121, 41, 2, 73, 3, 26, 14, 20, 33, 1758, 303, 4, 16, 75, 121, 14, 299, 15, 6, 153, 8, 112, 263, 18, 14, 20, 22, 96, 22, 16, 101, 219, 14, 21, 4, 12, 13, 9, 11, 12, 13, 9, 11, 61, 257, 7, 10886, 17974, 6, 15138, 13325, 4, 29, 18, 3, 42, 238, 3, 10, 45, 4874, 146, 272, 15138, 17974, 6, 62, 191, 7, 2, 17832, 4, 28, 2, 4570, 281, 206, 15, 2, 0, 6009, 5, 8198, 0, 8, 2, 19456, 3, 6, 257, 7, 0, 8062, 4, 12, 13, 9, 11, 12, 13, 9, 11, 14, 56, 39, 7, 163, 10355, 5908, 3, 2, 407, 357, 1437, 53, 1201, 768, 2, 9732, 4, 12, 13, 9, 11, 12, 13, 9, 11, 19, 81, 196, 31, 275, 218, 40, 19, 3, 561, 1789, 695, 10872, 4, 34, 10, 48, 222, 4, 558, 23, 727, 3982, 4, 34, 20, 15, 41, 149, 1699, 116, 4, 12, 13, 9, 11, 12, 13, 9, 11, 121, 174, 25, 318, 4, 470, 3, 33, 19, 4562, 19, 2, 216, 4, 18, 20, 70, 62, 357, 4, 318, 3, 121, 6, 553, 526, 55, 1573, 16582, 49, 19, 2839, 7, 2, 7106, 4, 19]
216
0


In [21]:
print "Pad sequences (samples x time)"
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print 'X_train shape:', X_train.shape
print 'X_test shape:', X_test.shape

Pad sequences (samples x time)
X_train shape: (20000, 100)
X_test shape: (5000, 100)


In [22]:
print X_train[0]
print X_train.shape

[   17    10     2   257     7    25    18    69  4195  1513    16   121
    41     2    73     3    26    14    20    33  1758   303     4    16
    75   121    14   299    15     6   153     8   112   263    18    14
    20    22    96    22    16   101   219    14    21     4    12    13
     9    11    12    13     9    11    61   257     7 10886 17974     6
 15138 13325     4    29    18     3    42   238     3    10    45  4874
   146   272 15138 17974     6    62   191     7     2 17832     4    28
     2  4570   281   206    15     2     0  6009     5  8198     0     8
     2 19456     3     6]
(20000, 100)


In [23]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 256))
model.add(LSTM(256, 128)) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(128, 1))
model.add(Activation('sigmoid'))

Build model...


In [24]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")

/Users/dikien/anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [25]:
print "Train..."
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=5, validation_split=0.1, show_accuracy=True)
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print 'Test score:', score

Train...
Train on 18000 samples, validate on 2000 samples
Epoch 0
18000/18000 [==============================] - 724s - loss: 0.6319 - acc.: 0.6557 - val. loss: 0.5908 - val. acc.: 0.7090
Epoch 1
18000/18000 [==============================] - 729s - loss: 0.4278 - acc.: 0.8183 - val. loss: 0.4212 - val. acc.: 0.8130
Epoch 2
18000/18000 [==============================] - 713s - loss: 0.2875 - acc.: 0.8902 - val. loss: 0.3895 - val. acc.: 0.8430
Epoch 3
18000/18000 [==============================] - 706s - loss: 0.2500 - acc.: 0.9041 - val. loss: 0.4578 - val. acc.: 0.8185
Epoch 4
5000/5000 [==============================] - 19s - loss: 0.5982    
Test score: 0.598056177763


In [26]:
classes = model.predict_classes(X_test, batch_size=batch_size)
acc = np_utils.accuracy(classes, y_test)
print 'Test accuracy:', acc

5000/5000 [==============================] - 21s    
Test accuracy: 0.809
